# Requirements

In [1]:
import math

# Problem setting


Jewish rebels were under siege in the Massada fortress by the Romans.  Since they could not hold out any longer, would not surrender, and their religion prohibits suicide, they came up with a creative solution.

The rebels stand in a circle.  The first rebel kill  the second, the third kills the fourth, and so on.  In general, the rebel standing clockwise from the one who was just killed, kills the next one in the cricle (clockwise).  The last man standing will commit suicide.

Josephus would prefer to live, so he has to be the last man standing, so that he can surrender to the Romans.  Which position in the circle should he take when there are 41 rebels?

# Implementations

Several implementations are possible.

## Formula

In general, when there are $n$ rebels, this can be written as $2^a + \ell$ where $\ell < 2^a$.  The position of the last man standing is $2\ell + 1$.

In [5]:
def last_position(n):
    a = int(math.log2(n))
    return 2*(n - 2**a) + 1

We can use this function to test the various implementations.

In [20]:
def test_kill_method(kill_method, max_rebels=42):
    for nr_rebels in range(1, max_rebels + 1):
        if kill_method(nr_rebels) != last_position(nr_rebels):
            print(f'Problem for {nr_rebels}: {kill_method(nr_rebels):3d} versus {last_position(nr_rebels):3d}')

## Classic implementation

The group of rebels is modeled as a list, from which the element that represents the victim is removed.  An index keeps track of the rebel that will be the next killer.  When there is only a single rebel left, the problem is solved.

In [2]:
def kill_rebels(n, is_animated=False):
    rebels = list(range(1, n + 1))
    if is_animated:
        print(rebels)
    killer_pos = 0
    while len(rebels) > 1:
        victim_pos = (killer_pos + 1) % len(rebels)
        rebels.pop(victim_pos)
        if is_animated:
            print(rebels)
        killer_pos = victim_pos
    return rebels[0]

Testing the implementation shows that it works well.

In [3]:
kill_rebels(5, is_animated=True)

[1, 2, 3, 4, 5]
[1, 3, 4, 5]
[1, 3, 5]
[3, 5]
[3]


3

In [21]:
test_kill_method(kill_rebels)

## Recursive approach


There is a recursive approach that doesn't require modulo operations to keep track of the index of the killer.  When the group of remaining rebels is viewed as a circle, in each step, the circle shrinks with one.  Also, if the new circle always has the next killer on top, it is really easy to keep track.

In [23]:
def kill_rebels_recursively(n):
    rebels = list(range(1, n + 1))
    def _kill_rebels(rebels):
        if len(rebels) == 1:
            return rebels[0]
        else:
            killer = rebels.pop(0)
            rebels.pop(0)
            rebels.append(killer)
            return _kill_rebels(rebels)
    return _kill_rebels(rebels)

In [24]:
kill_rebels_recursively(41)

19

In [26]:
test_kill_method(kill_rebels_recursively)

## Iterative implementation

This same algorithm is easy to implement iteratively as well.

In [11]:
def kill_rebels_iteratively(n):
    rebels = list(range(1, n + 1))
    while len(rebels) > 1:
        killer = rebels.pop(0)
        rebels.pop(0)
        rebels.append(killer)
    return rebels[0]

In [27]:
test_kill_method(kill_rebels_iteratively)

## Sieve implementation

An alternative implementation removes all dead rebels for a single traversal of the circle.  When that is done, we remove the first element of the remaining list if the length of the original circle length was odd, since the first member will certainly be killed in the next round.  Again, we are done when the list has a single element only.

In [28]:
def kill_rebels_sieve(n):
    rebels = list(range(1, n + 1))
    while True:
        is_odd = len(rebels) % 2 == 1
        rebels = rebels[::2]
        if len(rebels) == 1:
            return rebels[0]
        elif is_odd:
            rebels.pop(0)

In [29]:
test_kill_method(kill_rebels_sieve)

# Performance

In [34]:
%timeit kill_rebels(nr_rebels)

12.7 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [35]:
%timeit kill_rebels_recursively(nr_rebels)

20.4 µs ± 3.06 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [36]:
%timeit kill_rebels_iteratively(nr_rebels)

15.4 µs ± 909 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [37]:
%timeit kill_rebels_sieve(nr_rebels)

2.65 µs ± 245 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Results are not unexpected.  The classic implementation is faster than the recursive algorithm, as well as its iterative counterpart.  However, the sieve turns out the have the best performance.  This should not come as a surprise.  Although each iteration is fairly expensive, i.e., constructing and copying a new list, the number of iterations is $O(\log_2 n)$, rather than $O(n)$ as for the other implementations, and that outweighs the cost per iteration.